In [1]:
#Instacart Market Basket Analysis 5 - Feature Extraction

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import category_encoders as ce

root = '../instacart_data_files/'

In [72]:
aisles = pd.read_csv(root + 'aisles.csv')

departments = pd.read_csv(root + 'departments.csv')

orders = pd.read_csv(root + 'orders.csv', 
                 dtype={
                        'order_id': np.int32,
                        'user_id': np.int64,
                        'eval_set': 'category',
                        'order_number': np.int16,
                        'order_dow': np.int8,
                        'order_hour_of_day': np.int8,
                        'days_since_prior_order': np.float32})

order_products_prior = pd.read_csv(root + 'order_products__prior.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

order_products_train = pd.read_csv(root + 'order_products__train.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})
products = pd.read_csv(root + 'products.csv')


In [73]:
#Preparing Data

In [74]:
prior_df = order_products_prior.merge(orders, on ='order_id', how='inner')
prior_df = prior_df.merge(products, on = 'product_id', how = 'left')
prior_df.head()


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


In [75]:
#Features creation

In [76]:
#Calculating how many times a user buys the product

In [77]:
prior_df['user_buy_product_times'] = prior_df.groupby(['user_id', 'product_id']).cumcount() + 1
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [78]:
"""Product level features
(1) Product's average add-to-cart-order

(2) Total times the product was ordered

(3) Total times the product was reordered

(4) Reorder percentage of a product

(5) Total unique users of a product

(6) Is the product Organic?

(7) Percentage of users that buy the product second time"""

"Product level features\n(1) Product's average add-to-cart-order\n\n(2) Total times the product was ordered\n\n(3) Total times the product was reordered\n\n(4) Reorder percentage of a product\n\n(5) Total unique users of a product\n\n(6) Is the product Organic?\n\n(7) Percentage of users that buy the product second time"

In [79]:
# agg_dict1 = {'add_to_cart_order' : {'mean_add_to_cart_order':'mean'}, 
#            'reordered' : {'total_orders':'count', 'total_reorders':'sum', 'reorder_percentage':'mean'},
#            'user_id': {'unique_users' :lambda x: x.nunique()},
#            'user_buy_product_times': {'order_first_time_total_cnt' : lambda x: sum(x==1), 
#                                       'order_second_time_total_cnt' :lambda x: sum(x==2)},
#            'product_name':{'is_organic': lambda x: 1 if 'Organic' in x else 0}}

In [81]:
prod_feats1 = prior_df.groupby('product_id').agg(
    mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    total_orders=('reordered', 'count'),
    total_reorders=('reordered', 'sum'),
    reorder_percentage=('reordered', 'mean'),
    unique_users=('user_id', 'nunique'),
    order_first_time_total_cnt=('user_buy_product_times', lambda x: sum(x == 1)),
    order_second_time_total_cnt=('user_buy_product_times', lambda x: sum(x == 2)),
    is_organic=('product_name', lambda x: 1 if 'Organic' in x else 0)
)
prod_feats1.reset_index(inplace=True)
prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic
0,1,5.801836,1852,1136,0.613391,716,716,276,0
1,2,9.888889,90,12,0.133333,78,78,8,0
2,3,6.415162,277,203,0.732852,74,74,36,0
3,4,9.507599,329,147,0.446809,182,182,64,0
4,5,6.466667,15,9,0.600000,6,6,4,0


In [82]:
prod_feats1['second_time_percent'] = prod_feats1.order_second_time_total_cnt/prod_feats1.order_first_time_total_cnt


In [83]:
"""
Aisle and department features
(8) Reorder percentage, Total orders and reorders of a product aisle

(9) Mean and std of aisle add-to-cart-order

(10) Aisle unique users
"""

'\nAisle and department features\n(8) Reorder percentage, Total orders and reorders of a product aisle\n\n(9) Mean and std of aisle add-to-cart-order\n\n(10) Aisle unique users\n'

In [84]:
# agg_dict2 = {'add_to_cart_order' : {'aisle_mean_add_to_cart_order':'mean',
#                                    'aisle_std_add_to_cart_order':'std'}, 
#            'reordered' : {'aisle_total_orders':'count', 'aisle_total_reorders':'sum', 'aisle_reorder_percentage':'mean'},
#            'user_id': {'aisle_unique_users' :lambda x: x.nunique()}}


In [85]:
aisle_feats = prior_df.groupby('aisle_id').agg(
    aisle_mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    aisle_std_add_to_cart_order=('add_to_cart_order', 'std'),
    aisle_total_orders=('reordered', 'count'),
    aisle_total_reorders=('reordered', 'sum'),
    aisle_reorder_percentage=('reordered', 'mean'),
    aisle_unique_users=('user_id', 'nunique'),
)
aisle_feats.reset_index(inplace = True)
aisle_feats.head()


,aisle_id,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,aisle_total_reorders,aisle_reorder_percentage,aisle_unique_users
0,1,8.167640,7.104166,71928,42912,0.596597,20711
1,2,9.275497,7.473802,82491,40365,0.489326,31222
2,3,9.571935,7.899672,456386,272922,0.598007,63592
3,4,10.161450,7.745705,200687,98243,0.489533,53892
4,5,10.297600,8.187047,62510,17542,0.280627,32312


In [86]:
"""
features

(10) Reorder percentage, Total orders and reorders of a product department

(11) Mean and std of department add-to-cart-order

(12) Department unique users

"""

'\nfeatures\n\n(10) Reorder percentage, Total orders and reorders of a product department\n\n(11) Mean and std of department add-to-cart-order\n\n(12) Department unique users\n\n'

In [87]:
# agg_dict3 = {'add_to_cart_order' : {'department_mean_add_to_cart_order':'mean',
#                                    'department_std_add_to_cart_order':'std'}, 
#            'reordered' : {'department_total_orders':'count', 'department_total_reorders':'sum',
#                           'department_reorder_percentage':'mean'},
#            'user_id': {'department_unique_users' :lambda x: x.nunique()}}


In [88]:
# dpt_feats = prior_df.groupby('department_id').agg(agg_dict3)
dpt_feats = prior_df.groupby('department_id').agg(
    department_mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    department_std_add_to_cart_order=('add_to_cart_order', 'std'),
    department_total_orders=('reordered', 'count'),
    department_total_reorders=('reordered', 'sum'),
    department_reorder_percentage=('reordered', 'mean'),
    department_unique_users=('user_id', 'nunique'),
)
dpt_feats.reset_index(inplace = True)
dpt_feats.head()


,department_id,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users
0,1,8.996414,7.393502,2236432,1211890,0.541885,163233
1,2,8.277645,7.526272,36291,14806,0.407980,17875
2,3,8.084397,6.904849,1176787,739188,0.628141,140612
3,4,8.022875,6.658899,9479291,6160710,0.649913,193237
4,5,5.428346,5.778253,153696,87595,0.569924,15798


In [89]:
"""
features

(13) Binary encoding of aisle feature

(14) Binary encoding of department feature
"""

'\nfeatures\n\n(13) Binary encoding of aisle feature\n\n(14) Binary encoding of department feature\n'

In [90]:
prod_feats1 = prod_feats1.merge(products, on = 'product_id', how = 'left')
prod_feats1 = prod_feats1.merge(aisle_feats, on = 'aisle_id', how = 'left')
prod_feats1 = prod_feats1.merge(aisles, on = 'aisle_id', how = 'left')
prod_feats1 = prod_feats1.merge(dpt_feats, on = 'department_id', how = 'left')
prod_feats1 = prod_feats1.merge(departments, on = 'department_id', how = 'left')
prod_feats1.head()


,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,aisle_reorder_percentage,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department
0,1,5.801836,1852,1136,0.613391,716,716,276,0,0.385475,...,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973,0.574180,174219,snacks
1,2,9.888889,90,12,0.133333,78,78,8,0,0.102564,...,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301,0.346721,172755,pantry
2,3,6.415162,277,203,0.732852,74,74,36,0,0.486486,...,0.527615,53197,tea,6.976699,6.711172,2690129,1757892,0.653460,172795,beverages
3,4,9.507599,329,147,0.446809,182,182,64,0,0.351648,...,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890,0.541885,163233,frozen
4,5,6.466667,15,9,0.600000,6,6,4,0,0.666667,...,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301,0.346721,172755,pantry


In [91]:
prod_feats1.drop(['product_name', 'aisle_id', 'department_id'], axis = 1, inplace = True)
prod_feats1.head()


,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,aisle_reorder_percentage,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department
0,1,5.801836,1852,1136,0.613391,716,716,276,0,0.385475,...,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973,0.574180,174219,snacks
1,2,9.888889,90,12,0.133333,78,78,8,0,0.102564,...,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301,0.346721,172755,pantry
2,3,6.415162,277,203,0.732852,74,74,36,0,0.486486,...,0.527615,53197,tea,6.976699,6.711172,2690129,1757892,0.653460,172795,beverages
3,4,9.507599,329,147,0.446809,182,182,64,0,0.351648,...,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890,0.541885,163233,frozen
4,5,6.466667,15,9,0.600000,6,6,4,0,0.666667,...,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301,0.346721,172755,pantry


In [92]:
prod_feats1.shape


(49677, 24)

In [93]:
prod_feats1.dtypes

product_id                            uint16
mean_add_to_cart_order               float64
total_orders                           int64
total_reorders                         int64
reorder_percentage                   float64
unique_users                           int64
order_first_time_total_cnt             int64
order_second_time_total_cnt            int64
is_organic                             int64
second_time_percent                  float64
aisle_mean_add_to_cart_order         float64
aisle_std_add_to_cart_order          float64
aisle_total_orders                     int64
aisle_total_reorders                   int64
aisle_reorder_percentage             float64
aisle_unique_users                     int64
aisle                                 object
department_mean_add_to_cart_order    float64
department_std_add_to_cart_order     float64
department_total_orders                int64
department_total_reorders              int64
department_reorder_percentage        float64
department

In [94]:
encoder= ce.BinaryEncoder(cols=['aisle', 'department'],return_df=True)


In [95]:
prod_feats1 = encoder.fit_transform(prod_feats1)
prod_feats1.head()


,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136,0.613391,716,716,276,0,0.385475,...,7.692492,2887550,1657973,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12,0.133333,78,78,8,0,0.102564,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203,0.732852,74,74,36,0,0.486486,...,6.711172,2690129,1757892,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147,0.446809,182,182,64,0,0.351648,...,7.393502,2236432,1211890,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9,0.600000,6,6,4,0,0.666667,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0


In [96]:
prod_feats1.shape


(49677, 35)

In [97]:
prod_feats1.columns


Index(['product_id', 'mean_add_to_cart_order', 'total_orders',
       'total_reorders', 'reorder_percentage', 'unique_users',
       'order_first_time_total_cnt', 'order_second_time_total_cnt',
       'is_organic', 'second_time_percent', 'aisle_mean_add_to_cart_order',
       'aisle_std_add_to_cart_order', 'aisle_total_orders',
       'aisle_total_reorders', 'aisle_reorder_percentage',
       'aisle_unique_users', 'aisle_0', 'aisle_1', 'aisle_2', 'aisle_3',
       'aisle_4', 'aisle_5', 'aisle_6', 'aisle_7',
       'department_mean_add_to_cart_order', 'department_std_add_to_cart_order',
       'department_total_orders', 'department_total_reorders',
       'department_reorder_percentage', 'department_unique_users',
       'department_0', 'department_1', 'department_2', 'department_3',
       'department_4'],
      dtype='object')

In [98]:
prod_feats1.isnull().any().any()

False

In [99]:
# free some memory
del aisle_feats, dpt_feats, aisles, departments
gc.collect()

0

In [100]:
"""
User level features
(15) User's average and std day-of-week of order

(16) User's average and std hour-of-day of order

(17) User's average and std days-since-prior-order

(18) Total orders by a user

(19) Total products user has bought

(20) Total unique products user has bought

(21) user's total reordered products

(22) User's overall reorder percentage

"""

"\nUser level features\n(15) User's average and std day-of-week of order\n\n(16) User's average and std hour-of-day of order\n\n(17) User's average and std days-since-prior-order\n\n(18) Total orders by a user\n\n(19) Total products user has bought\n\n(20) Total unique products user has bought\n\n(21) user's total reordered products\n\n(22) User's overall reorder percentage\n\n"

In [101]:
prior_df.isnull().any()

order_id                  False
product_id                False
add_to_cart_order         False
reordered                 False
user_id                   False
eval_set                  False
order_number              False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
product_name              False
aisle_id                  False
department_id             False
user_buy_product_times    False
dtype: bool

In [102]:
# when no prior order, the value is null. Imputing as 0
prior_df.days_since_prior_order = prior_df.days_since_prior_order.fillna(0)

In [103]:
prior_df.head()


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [104]:
agg_dict4 = {'order_dow': {'avg_dow':'mean', 'std_dow':'std'},
           'order_hour_of_day': {'avg_doh':'mean', 'std_doh':'std'},
           'days_since_prior_order': {'avg_since_order':'mean', 'std_since_order':'std'},
           'order_number': {'total_orders_by_user': lambda x: x.nunique()},
           'product_id': {'total_products_by_user': 'count',
                         'total_unique_product_by_user': lambda x: x.nunique()},
           'reordered': {'total_reorders_by_user':'sum', 
                        'reorder_propotion_by_user':'mean'}}

In [105]:
# user_feats = prior_df.groupby('user_id').agg(agg_dict4)

user_feats = prior_df.groupby('user_id').agg(
    avg_dow=('order_dow', 'mean'),
    std_dow=('order_dow', 'std'),
    avg_doh=('order_hour_of_day', 'mean'),
    std_doh=('order_hour_of_day', 'std'),
    avg_since_order=('days_since_prior_order', 'mean'),
    std_since_order=('days_since_prior_order', 'std'),
    total_orders_by_user=('order_number', 'nunique'),
    total_products_by_user=('product_id', 'count'),
    total_unique_product_by_user=('product_id', 'nunique'),
    total_reorders_by_user=('reordered', 'sum'),
    reorder_propotion_by_user=('reordered', 'mean')
)
user_feats.reset_index(inplace = True)
user_feats.head()


,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559065,10,59,18,41,0.694915
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378


In [106]:
"""
features

(23) Average order size of a user

(24) User's mean of reordered items of all orders
"""

"\nfeatures\n\n(23) Average order size of a user\n\n(24) User's mean of reordered items of all orders\n"

In [107]:
agg_dict5 = {'reordered': {'average_order_size':'count', 
                        'reorder_in_order':'mean'}}

user_feats2 = prior_df.groupby('user_id').agg(
    average_order_size=('reordered', 'count'),
    reorder_in_order=('reordered', 'mean')
)

user_feats2.reset_index(inplace = True)
user_feats2.head()


,user_id,average_order_size,reorder_in_order
0,1,59,0.694915
1,2,195,0.476923
2,3,88,0.625000
3,4,18,0.055556
4,5,37,0.378378


In [108]:
user_feats3 = user_feats2.groupby('user_id').agg({'average_order_size' : 'mean', 
                                   'reorder_in_order':'mean'})
user_feats3 = user_feats3.reset_index()
user_feats3.head()

,user_id,average_order_size,reorder_in_order
0,1,59.0,0.694915
1,2,195.0,0.476923
2,3,88.0,0.625000
3,4,18.0,0.055556
4,5,37.0,0.378378


In [109]:
user_feats = user_feats.merge(user_feats3, on = 'user_id', how = 'left')
user_feats.head()


,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559065,10,59,18,41,0.694915,59.0,0.694915
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,195.0,0.476923
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,88.0,0.625000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556,18.0,0.055556
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,37.0,0.378378


In [110]:
"""
features

(25) Percentage of reordered itmes in user's last three orders

(26) Total orders in user's last three orders

Last 3 orders of a user

"""

"\nfeatures\n\n(25) Percentage of reordered itmes in user's last three orders\n\n(26) Total orders in user's last three orders\n\nLast 3 orders of a user\n\n"

In [111]:
last_three_orders = user_feats2.groupby('user_id')['order_number'].nlargest(3).reset_index()
last_three_orders.head()

KeyError: 'Column not found: order_number'